In [1]:
import heapq
import numpy as np
import pandas as pd
import pyproj
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from math import sqrt, pi, sin, cos

wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator
washington_y = 38.904978
washington_x = -77.039658



"""this simple little script just takes the index and makes a unique key"""
# index_key = lambda t, m: (t[0]+m+1 + (t[1]+m+1)*2*m)

class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]


class path_finder:
    def __init__(self, grid_rings):
        self.hex_meters = 800.0
        self.grid_rings = grid_rings
        self.X, self.Y = pyproj.transform(wgs84, webdb, washington_x, washington_y)
        
        self.grid_creation()
        self.neighbors_ring = self.make_ring(1)
        self.neighbors = tuple(self.key_lookup(r[0],r[1])-self.key_lookup(0,0) for r in self.neighbors_ring)
        
        self.stations = pd.read_pickle('subway_locations')
        
        YX = self.hexes['grid'].progress_apply(self.yx_generator)
        self.hexes['Y'] = YX.apply(lambda x: x[0])
        self.hexes['X'] = YX.apply(lambda x: x[1])
        
        self.hexes['cost'] = None

    def key_lookup(self, x, y):
        m = self.grid_rings
        return (x + m) + (y+m)*(1+2*m)
    
    def station_locator(self):
        
        pyproj.transform(wgs84, webdb, row[2], row[1])
    
    def make_ring(self,n):
        """All the hexagons n hexes from the center"""
        ring = []
        for m in range(1,n+1):
            ring.append((n,-m,-n+m))
            ring.append((-m,-n+m,n))
            ring.append((-n+m,n,-m))
            ring.append((-n,m, n-m))
            ring.append((m,n-m,-n))
            ring.append((n-m,-n,m))
        return ring
    
    def fill_grid(self):
        hex_list = [(0,0,0)]
        for n in range(1,self.grid_rings+1):
            hex_list+=self.make_ring(n)
        return hex_list
        
    def yx_generator(self, grid):
        """Bokeh expects the coordinates to be passed in a list of x coordinates
        and a seperate list of y coordinates.  This generates those for each hex"""
        size = self.hex_meters
        cy = size * sqrt(3) * ( grid[1] + grid[0]/2 ) + self.Y
        cx = 1.5 * float(grid[0]) * size + self.X
        Y=[]
        X=[]
        for n in range(6):
            angle=n*pi/3
            Y.append(size*sin(angle)+cy)            
            X.append(size*cos(angle)+cx)
        return Y, X

    def grid_creation(self):
        """This creates the dataframe with all the hex grid information."""
        g = self.fill_grid()
        l = len(g)
        s = self.grid_rings
        """The number of hexagons from center to edge is stored at s,
        it will be used when calculating the 'index_key'.  
        This is a unique value for every hex and needs to use s to
        make sure there aren't any overlaps"""

        self.hexes = pd.DataFrame(np.zeros((l,1)), columns=['grid'])
        self.hexes['grid'] = g
        self.hexes['hex_key'] = self.hexes.grid.progress_apply(lambda x: self.key_lookup(x[0],x[1]))
        self.hexes.set_index('hex_key', inplace=True)

    def get_valid_neighbors(self, key):
            return tuple((key + n) for n in self.neighbors if (key+n) in self.index_)

    def check_if_better(self, new_loc, from_cost, from_hex):
        old_cost, next_weight, next_source = self.hexes.loc[new_loc,['cost','weight', 'source']]
        
        if(type(next_weight))==str:
            print(self.hexes.loc[new_loc])
        new_cost = next_weight + from_cost
        if (old_cost is None) or (new_cost < old_cost):
            self.hexes.at[new_loc,['cost', 'source']] = new_cost, from_hex
            self.frontier.put(new_loc, new_cost)
        
    def dijkstra_map(self, origin):
        input_df = self.hexes
        start = self.key_lookup(origin[0],origin[1])

        self.index_ = set(input_df.index)
        input_df['source'] = None
#         input_df['cost'] = None
        input_df['weight'] = int(1)
        input_df.at[start, 'source'] = 0
        input_df.at[start, 'cost'] = 0
        self.frontier = PriorityQueue()    
        self.frontier.put(start, 0)

        while not self.frontier.empty():
            current_hex = self.frontier.get()
#             print(type(current_hex))
            current_cost = input_df.loc[current_hex].cost
            if(current_cost is None):
                print('error: no cost recorded for visited cell')
                break
            neighbors = self.get_valid_neighbors(current_hex)

            for i in neighbors:
                self.check_if_better(i, current_cost, current_hex)



In [2]:
# stations = pd.read_pickle('stations2.pickle')
# stations[['hex','hex_key']].to_pickle('stations3.pickle')
stations = pd.read_pickle('stations3.pickle')

In [3]:
new_map = path_finder(30)

for hex, station in tqdm_notebook(zip(stations.hex.values, stations.hex_key.values), total=stations.shape[0]):
    new_map.dijkstra_map((hex[0],hex[1],hex[2]))

HBox(children=(IntProgress(value=0, max=2791), HTML(value='')))

HBox(children=(IntProgress(value=0, max=2791), HTML(value='')))

In [4]:
new_map.hexes.to_pickle('pathfound_pickle')
new_map.hexes.head()

,grid,Y,X,cost
hex_key,,,,
1860,"(0, 0, 0)","[4708069.611732003, 4708762.432055031, 4708762...","[-8575215.499447944, -8575615.499447944, -8576...",None
1800,"(1, -1, 0)","[4707376.791408976, 4708069.611732003, 4708069...","[-8574015.499447944, -8574415.499447944, -8575...",None
1859,"(-1, 0, 1)","[4707376.791408976, 4708069.611732003, 4708069...","[-8576415.499447944, -8576815.499447944, -8577...",None
1921,"(0, 1, -1)","[4709455.252378059, 4710148.072701086, 4710148...","[-8575215.499447944, -8575615.499447944, -8576...",None
1920,"(-1, 1, 0)","[4708762.432055031, 4709455.252378059, 4709455...","[-8576415.499447944, -8576815.499447944, -8577...",None
